In [12]:
%pylab inline

import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import datetime, time

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [13]:
# load train and test data
wnv_train = np.loadtxt('../working/train_f.csv', delimiter=',', dtype=float64, skiprows=1)
wnv_test = np.loadtxt('../working/test_f.csv', delimiter=',', dtype=float64, skiprows=1)

In [14]:
# raw data
yr_train = wnv_train[:,1].astype(int)
Xr_train = wnv_train[:,2:]
Xr_test = wnv_test[:,1:]

# normalize data
scaler = StandardScaler()
scaler.fit(Xr_train)
Xn_train = scaler.transform(Xr_train)
Xn_test = scaler.transform(Xr_test)

X_train, X_test, y_train, y_test = train_test_split(Xn_train, yr_train)

In [23]:
# parameters
n_neighbors = 3
n_components = 65
n_estimators = 1000
n_jobs = 3
p_level = 0.3

In [25]:
#clf = KNeighborsClassifier(n_neighbors=n_neighbors)
clf = ExtraTreesClassifier(n_estimators=n_estimators, n_jobs=n_jobs)
#clf.fit(X_train, y_train)
clf.fit(Xn_train, yr_train)

# use the model to predict the labels of the test data
predicted = clf.predict(X_test)
proba = clf.predict_proba(X_test)
expected = y_test

In [26]:
# accuracy
matches = (predicted == expected)
print matches.sum() / float(len(matches))

# f1 score
from sklearn import metrics
print metrics.classification_report(expected, predicted)

# confusion matrix
print metrics.confusion_matrix(expected, predicted)

# auc
print metrics.roc_auc_score(expected, proba[:,1])

0.980966882375
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      2489
          1       0.98      0.65      0.78       138

avg / total       0.98      0.98      0.98      2627

[[2487    2]
 [  48   90]]
0.994838157458


In [27]:
print np.count_nonzero(predicted == 1)
print np.count_nonzero(y_test)

92
138


In [28]:
proba = clf.predict_proba(Xn_test)
print np.max(proba[:,1])

0.607352380952


In [30]:
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')

def write_preds(proba, fname):
    pd.DataFrame({"Id": list(range(1,len(proba)+1)), "WnvPresent": proba}).to_csv(fname, index=False, header=True)

#write_preds(preds[range(28000)], "keras-mlp_" + st + ".csv")
write_preds(proba[:,1], "../working/sklearn_test_" + st + ".csv")